* RAG = retrieval augmented generation

RAG PipeLine for this project 

1. Load Data from different source (Pdf,Excel,Text,Json)
2. Load Data 
3. Transform 
4. embeding
5. store in vector Db 
6. embed the input 
7. retrieve  most similar content 

* Data Ingestion 
document loaders to load data from a source as Document's. 
A Document is a piece of text and associated metadata

In [25]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader('/home/mshameem/Documents/Generative_Ai/Simple RAG Project/Atomic_habit.txt')
text_document = loader.load()
docs= text_document

In [32]:
from dotenv import load_dotenv
import os 
load_dotenv()
hugging_Api = os.getenv('Hugging_Face_API')
pinecone_API = os.getenv('pinecone_Api')

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

# Define the URL to load
url = 'https://forwardfitnessstl.com/2021/01/book-review-atomic-habits-by-james-clear/'

# Use SoupStrainer to filter specific parts of the HTML
bs_kwargs = dict(
    parse_only=bs4.SoupStrainer(class_=['post-title','post-content','post-header'])
)

# Initialize the WebBaseLoader
loader = WebBaseLoader(web_path=url, bs_kwargs=bs_kwargs)

# Load the documents
text_documents = loader.load()


In [24]:
text_documents

[Document(metadata={'source': 'https://forwardfitnessstl.com/2021/01/book-review-atomic-habits-by-james-clear/'}, page_content='')]

In [26]:
# transform
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(docs)

In [28]:
documents[:1]

[Document(metadata={'source': '/home/mshameem/Documents/Generative_Ai/Simple RAG Project/Atomic_habit.txt'}, page_content='What My Intention Is" based on the Atomic Habits approach. I remember \nreading about this method where you break down your intentions into tiny, \nactionable steps called microhabits. Each of these microhabits is supposed \nto help build up to a larger habit over time.\n\nFirst, I think I should start by understanding what exactly an atomic \nhabit is. From what I recall, it\'s the smallest possible step toward achieving a bigger goal or habit. These microactions are designed to be so small and specific that you can \ncommit to doing them without hesitation. So, if my main intention is to "write every day," a microhabit might be something like writing for just five minutes each morning.\n\nBut wait, I want the text in the file to outline my intentions at various levels: my overall intention, major habits, smaller habits, and down to individual actions. How do I \n

In [38]:
# embeding
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
# hugging face embeding 
embeddings = HuggingFaceInferenceAPIEmbeddings(api_key=hugging_Api,model_name="sentence-transformers/all-mpnet-base-v2")
# pinecone 
index_name = 'learnlangchin'
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings,pinecone_api_key=pinecone_API)


In [39]:
vectorstore.add_documents(docs)

['acb2c0fd-911c-4d17-aea2-e1609f05f0da']

In [44]:
query = 'Atomic Habits'
result=vectorstore.similarity_search(query)
result[0].page_content

'What My Intention Is" based on the Atomic Habits approach. I remember \nreading about this method where you break down your intentions into tiny, \nactionable steps called microhabits. Each of these microhabits is supposed \nto help build up to a larger habit over time.\n\nFirst, I think I should start by understanding what exactly an atomic \nhabit is. From what I recall, it\'s the smallest possible step toward achieving a bigger goal or habit. These microactions are designed to be so small and specific that you can \ncommit to doing them without hesitation. So, if my main intention is to "write every day," a microhabit might be something like writing for just five minutes each morning.\n\nBut wait, I want the text in the file to outline my intentions at various levels: my overall intention, major habits, smaller habits, and down to individual actions. How do I \nstructure this?\n\nMaybe I should begin by listing my main intention. That\'s straightforward—just one line about what I\'